In [ ]:
from dataclasses import dataclass
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

In [ ]:
class TensorField:
    def __init__(self, N, M):
        self.N = N
        self.M = M
        self.base_fields = []

        self.field = None
        self.eigen_vectors = None

    def add_grid_basis(self, direction = np.array([1., 1.]), origin=np.array([0, 0])):
        l = np.linalg.norm(direction)
        angle = np.arctan(direction[1] / (direction[0] + 1e-4))
        tensor = l * np.array([
            [np.cos(2 * angle), np.sin(2 * angle)],
            [np.sin(2 * angle), -np.cos(2 * angle)]
        ], dtype=np.float32)
        self.base_fields.append(
            {
                "type": "grid",
                "origin": origin,
                "tensor": lambda p: tensor
            }
        )

    def add_radial_basis(self, origin=np.array([0, 0])):
        origin = origin.astype(np.float32)
        self.base_fields.append(
            {
                "type": "radial",
                "origin": origin.astype(np.float32),
                "tensor": lambda p: np.array([
                    [(p[1] - origin[1]) ** 2 - (p[0] - origin[0]) ** 2, 
                     -2 * (p[0] - origin[0]) * (p[1] - origin[1])],
                    [-2 * (p[0] - origin[0]) * (p[1] - origin[1]), 
                     (origin[0] - p[0]) ** 2 - (origin[1] - p[1]) ** 2]
                ], dtype=np.float32)
            }
        )
    def add_line(self, line, closed = False, sample_step=None):
        n_points = len(line)
        for i in range(n_points):
            cur_point = line[i]
            if i == n_points - 1:
                if closed:
                    next_point = line[0]
                else:
                    break
            else:
                next_point = line[i + 1]

            vec = next_point - cur_point
            self.add_grid_basis(vec, cur_point)
            
            if sample_step is not None:
                while np.linalg.norm(vec) > sample_step:
                    cur_point = cur_point + vec / np.linalg.norm(vec) * sample_step
                    vec = next_point - cur_point
                    self.add_grid_basis(vec, cur_point)
            
                

    def calculate_field(self, recalculate = False, decay=0.1):
        if recalculate or self.field is None or self.eigen_vectors is None:
            assert len(self.base_fields) > 0, "No basis fields"
            zero_tensor = np.array([[0., 0.], [0., 0.]], dtype=np.float32)
            self.field = [[zero_tensor for _ in range(self.M)] for _ in range(self.N)]
            self.eigen_vectors = [[zero_tensor for _ in range(self.M)] for _ in range(self.N)]
            
            for i in range(self.N):
                for j in range(self.M):
                    cur_point = np.array([i, j], dtype=np.float32)
                    for basis_field_meta in self.base_fields:
                        weight = np.exp(-decay * np.linalg.norm(
                               cur_point - basis_field_meta["origin"]
                            ))
                        # weight = 1.
                        self.field[i][j] = self.field[i][j] + weight * basis_field_meta["tensor"](cur_point)

            self.field = np.array(self.field)

            for i in range(self.N):
                for j in range(self.M):
                    eigenvalues, eigenvectors = np.linalg.eig(self.field[i, j])
                    idxs = np.argsort(eigenvalues)[::-1]
                    self.eigen_vectors[i][j] = eigenvectors.T[idxs]

            self.eigen_vectors = np.array(self.eigen_vectors)

    def vis_field(self):
        X, Y = np.meshgrid(np.arange(self.N), np.arange(self.M))
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        U = self.eigen_vectors[:, :, 0, 0]
        V = self.eigen_vectors[:, :, 0, 1]
        U_minor = self.eigen_vectors[:, :, 1, 0]
        V_minor = self.eigen_vectors[:, :, 1, 1]
        # ax[0].axis('equal')
        ax[0].set_aspect('equal', adjustable='box')
        ax[0].set_xlim(0, self.N-1)
        ax[0].set_ylim(0, self.M-1)
        ax[0].quiver(X.T, Y.T, U, V, color='r')
        ax[0].quiver(X.T, Y.T, U_minor, V_minor, color='g')
        ax[0].quiver(X.T, Y.T, -U, -V, color='r')
        ax[0].quiver(X.T, Y.T, -U_minor, -V_minor, color='g')

        print(X.shape, Y.shape, U.shape, V.shape)
        seed_points = np.array([[10], [10]])
        # ax[1].axis('equal')
        ax[1].set_aspect('equal', adjustable='box')
        ax[1].set_xlim(0, self.N-1)
        ax[1].set_ylim(0, self.M-1)
        # ax[1].axis('equal')
        ax[1].streamplot(X, Y, U.T, V.T, density=2, color='r')#, start_points=seed_points.T)
        # ax[1].streamplot(X, Y, U_minor.T, V_minor.T, density=2, color='g')#, start_points=seed_points.T)
        return fig, ax 
        # plt.show()

        
        

In [ ]:
tf = TensorField(16, 20)
tf.add_grid_basis(direction = np.array([1, 0.5]))
tf.add_radial_basis(origin=np.array([15, 15]))
tf.calculate_field(decay=10.)
tf.vis_field()
plt.show()

In [ ]:
tf = TensorField(20, 20)
poly = np.array([[5, 5], [10, 2.5], [15.5, 7.5], [10, 17.5]])
tf.add_line(poly, closed=True, sample_step=1.)
tf.calculate_field(decay=12.)
tf.vis_field()
plt.fill(poly[:, 0], poly[:, 1], facecolor='skyblue', edgecolor='blue', linewidth=2)
plt.show()

In [ ]:
def my_curve_function(x):
    return x**2 * np.sin(x) / 300 * 5 + 10

x = np.linspace(0, 20, 100)  # Generate 100 points between -5 and 5
y = my_curve_function(x)
curve = np.array([x, y]).T
tf = TensorField(20, 20)
tf.add_line(curve, closed=False, sample_step=0.5)
tf.calculate_field(decay=12.)
tf.vis_field()
plt.plot(x, y)
plt.show()

In [ ]:
def get_random_quad(N, M, h=7, w=4):
    left_bot_x = np.random.uniform(0, N-1-h-1)
    left_bot_y = np.random.uniform(0, M-1-w-1)
    right_top_x = left_bot_x + h + np.random.randn()
    right_top_y = left_bot_y + w + np.random.randn()
    return np.array([[left_bot_x, left_bot_y], [right_top_x, left_bot_y],
                    [right_top_x, right_top_y], [left_bot_x, right_top_y]])

In [ ]:
tf = TensorField(20, 20)
boundary = np.array([[0, 0], [0, tf.M], [tf.N, tf.M], [tf.N, 0],])
polygon1 = get_random_quad(tf.N, tf.M)
# polygon2 = get_random_quad(tf.N, tf.M)
print(polygon1)
tf.add_line(boundary, closed=True, sample_step=1.)
tf.add_line(polygon1, closed=True, sample_step=1.)
# tf.add_line(polygon2, closed=True, sample_step=1.)
tf.calculate_field(decay=12.)
tf.vis_field()
plt.fill(polygon1[:, 0], polygon1[:, 1], facecolor='skyblue', edgecolor='blue', linewidth=2)
# plt.fill(polygon2[:, 0], polygon2[:, 1], facecolor='skyblue', edgecolor='blue', linewidth=2)
plt.show()

In [ ]:
tf = TensorField(10, 5)
boundary = np.array([[0, 0], [0, tf.M], [tf.N, tf.M], [tf.N, 0],])
# polygon1 = np.random(rand
# polygon2 = get_random_quad(tf.N, tf.M)
# print(polygon1)
tf.add_line(boundary, closed=True, sample_step=1.)
# tf.add_line(polygon1, closed=True, sample_step=1.)
# tf.add_line(polygon2, closed=True, sample_step=1.)
tf.calculate_field(decay=12.)
tf.vis_field()
# plt.fill(polygon1[:, 0], polygon1[:, 1], facecolor='skyblue', edgecolor='blue', linewidth=2)
# plt.fill(polygon2[:, 0], polygon2[:, 1], facecolor='skyblue', edgecolor='blue', linewidth=2)
plt.show()

In [ ]:
def get_curvature_map(tf, delta=1.):
    X, Y = np.meshgrid(np.arange(0, tf.N - 1, delta), np.arange(0, tf.M - 1, delta))
    grid_points = np.array([X.T.reshape((-1)), Y.T.reshape((-1))]).T

    X, Y = np.meshgrid(np.arange(tf.N), np.arange(tf.M))
    U = tf.eigen_vectors[:, :, 0, 0]
    V = tf.eigen_vectors[:, :, 0, 1]
    strm = plt.streamplot(X, Y, -U.T, -V.T, density=2, color='r', start_points=grid_points)
    plt.scatter(grid_points[:, 0], grid_points[:, 1])
    
    print(len(strm.lines.get_paths()), len(grid_points))
get_curvature_map(tf, delta=0.1)

In [ ]:
X, Y = np.meshgrid(np.arange(0, tf.N, 0.1), np.arange(0, tf.M, 0.1))

In [ ]:
X.T.shape

In [ ]:
np.array([X.T.reshape((-1)), Y.T.reshape((-1))]).T[-10:]

In [ ]:
start_points = np.array([[5.68, 1.  ]])
tf.eigen_vectors.shape

In [ ]:
X, Y = np.meshgrid(np.arange(tf.N), np.arange(tf.M))
points = np.vstack((X.T.reshape((-1)), Y.T.reshape((-1)))).T
U = tf.eigen_vectors[:, :, 0, 0]
V = tf.eigen_vectors[:, :, 0, 1]
u_interp = interpolate.griddata(points, U.reshape((-1)), (start_points[:, 0], start_points[:, 1]), method='cubic')
v_interp = interpolate.griddata(points, V.reshape((-1)), (start_points[:, 0], start_points[:, 1]), method='cubic')
u_interp, v_interp

In [ ]:
u_interp[0,0]

In [ ]:
fig, ax = tf.vis_field()
# plt.fill(polygon1[:, 0], polygon1[:, 1], facecolor='skyblue', edgecolor='blue', linewidth=2)
# plt.fill(polygon2[:, 0], polygon2[:, 1], facecolor='skyblue', edgecolor='blue', linewidth=2)
ax[0].arrow(start_point[0], start_point[1], u_interp[0], v_interp[0], width=0.05, head_width=0.2, head_length=0.3,)
ax[1].arrow(start_point[0], start_point[1], u_interp[0], v_interp[0], width=0.05, head_width=0.2, head_length=0.3,)
plt.show()

In [ ]:
tf.eigen_vectors[:, :, 0, 1].shape

In [ ]:
np.gradient([[1, 2, 4],
             [2, 3, 4]], axis=-1)

In [ ]:
a = strm.lines.get_paths()[0].vertices

In [ ]:
dx_dt = np.gradient(a[:, 0])
dy_dt = np.gradient(a[:, 1])
velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

In [ ]:
ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

In [ ]:
tangent = np.array([1/ds_dt] * 2).transpose() * velocity

In [ ]:
tangent_x = tangent[:, 0]
tangent_y = tangent[:, 1]

deriv_tangent_x = np.gradient(tangent_x)
deriv_tangent_y = np.gradient(tangent_y)

dT_dt = np.array([ [deriv_tangent_x[i], deriv_tangent_y[i]] for i in range(deriv_tangent_x.size)])

length_dT_dt = np.sqrt(deriv_tangent_x * deriv_tangent_x + deriv_tangent_y * deriv_tangent_y)

In [ ]:
d2s_dt2 = np.gradient(ds_dt)
d2x_dt2 = np.gradient(dx_dt)
d2y_dt2 = np.gradient(dy_dt)

curvature = np.abs(d2x_dt2 * dy_dt - dx_dt * d2y_dt2) / (dx_dt * dx_dt + dy_dt * dy_dt)**1.5

In [ ]:
plt.plot(curvature)

In [ ]:
idx = np.argwhere(np.isclose(strm.lines.get_paths()[0].vertices, start_point).sum(axis=-1) == 2)
curvature[idx[0, 0]]

In [ ]:
def

In [ ]:
X, Y = np.meshgrid(np.arange(tf.N), np.arange(tf.M))
U = tf.eigen_vectors[:, :, 0, 0]
V = tf.eigen_vectors[:, :, 0, 1]
strm = plt.streamplot(X, Y, -U.T, -V.T, density=2, color='r', start_points=start_points)
plt.scatter(start_points[:, 0], start_points[:, 1])

In [ ]:
@dataclass
class Shelf:
    x: float = 0
    y: float = 0
    l: float = 1.55
    w: float = 0.6
    orientation: str ='horizontal'
    occupancy_width: float = 0.2

    def get_polygon(self):
        if self.orientation == 'horizontal':            
            polygon = [
                [self.x - self.l / 2, self.y - self.w / 2],
                [self.x + self.l / 2, self.y - self.w / 2],
                [self.x + self.l / 2, self.y + self.w / 2],
                [self.x - self.l / 2, self.y + self.w / 2],         
            ]
            occupancy_polygon = [
                [polygon[0][0], polygon[0][1] - self.occupancy_width],
                [polygon[1][0], polygon[1][1] - self.occupancy_width],
                [polygon[2][0], polygon[2][1] + self.occupancy_width],
                [polygon[3][0], polygon[3][1] + self.occupancy_width]
            ]
        elif self.orientation == 'vertical':
            polygon = [
                [self.x - self.w / 2, self.y - self.l / 2],
                [self.x + self.w / 2, self.y - self.l / 2],
                [self.x + self.w / 2, self.y + self.l / 2],
                [self.x - self.w / 2, self.y + self.l / 2],         
            ]
            occupancy_polygon = [
                [polygon[0][0] - self.occupancy_width, polygon[0][1]],
                [polygon[1][0] + self.occupancy_width, polygon[1][1]],
                [polygon[2][0] + self.occupancy_width, polygon[2][1]],
                [polygon[3][0] - self.occupancy_width, polygon[3][1]]
            ]
        else:
            raise RuntimeError("Wrong orientation")
        return np.array(polygon), np.array(occupancy_polygon)

    def is_valid(self, size_x, size_y):
        polygon, occupancy_polygon = self.get_polygon()
        if np.any(occupancy_polygon[:, 0] > size_x) or np.any(occupancy_polygon[:, 0] < 0) or \
            np.any(occupancy_polygon[:, 1] > size_y) or np.any(occupancy_polygon[:, 1] < 0):
            return False
        return True

    def draw(self, axes, show_occupancy=True):
        polygon, occupancy_polygon = self.get_polygon()
        # if self.orientation == 'horizontal':
        #     facecolor='skyblue'
        #     edgecolor='blue'
        # else:
        #     facecolor='mistyrose'
        #     edgecolor='red'
        facecolor='skyblue'
        edgecolor='blue'
        if show_occupancy:
            axes.fill(occupancy_polygon[:, 0], occupancy_polygon[:, 1], facecolor='gray', edgecolor='black', linewidth=2)
        axes.fill(polygon[:, 0], polygon[:, 1], facecolor=facecolor, edgecolor=edgecolor, linewidth=2)

def check_overlap(l1, r1, l2, r2):
    if r2[0] < l1[0] or r1[0] < l2[0]:
        return False
    if r2[1] < l1[1] or r1[1] < l2[1]:
        return False
    return True

def check_shelfs_overlap(s1: Shelf, s2: Shelf):
    poly1, occup1 = s1.get_polygon()
    poly2, occup2 = s2.get_polygon()
    if check_overlap(poly1[0], poly1[2], occup2[0], occup2[2]):
        return True
    if check_overlap(occup1[0], occup1[2], poly2[0], poly2[2]):
        return True
    if s1.orientation != s2.orientation: # !!!
        if check_overlap(occup1[0], occup1[2], occup2[0], occup2[2]):
            return True
    return False

def check_collisions(new_shelf: Shelf, shelves_list: list):
    for shelf in shelves_list:
        if check_shelfs_overlap(new_shelf, shelf):
            return True
    return False
    
    
s1 = Shelf(x=3, y=3)
s2 = Shelf(x=4.3, y=2, l =3, orientation='vertical')
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
s1.draw(ax)
s2.draw(ax)
print(s1.is_valid(5, 4), s2.is_valid(8, 3))
print(check_shelfs_overlap(s1, s2))

In [ ]:
0.547 * 2

In [ ]:
# def get_major_eigen(point):

def is_horizontal(vec, thresh=0.2):
    unit_vec = vec / np.linalg.norm(vec)
    if 1 - np.abs(np.dot(unit_vec, [1, 0])) < thresh:
        return True
    return False

def is_vertical(vec, thresh=0.2):
    unit_vec = vec / np.linalg.norm(vec)
    if 1 - np.abs(np.dot(unit_vec, [0, 1])) < thresh:
        return True
    return False

def place_shelves(tf, start_point=np.array([1., 1.]), passage_width=0.5, shelf_l=1.55, shelf_w=1.084, occupancy_width=0.7, skip_shelf_prob=0., thresh=0.2):
    shelves = []
    cur_position = start_point.copy()

    X, Y = np.meshgrid(np.arange(tf.N), np.arange(tf.M))
    points = np.vstack((X.T.reshape((-1)), Y.T.reshape((-1)))).T
    U = tf.eigen_vectors[:, :, 0, 0]
    V = tf.eigen_vectors[:, :, 0, 1]
    
    def _get_major_eigen(p):
        u_interp = interpolate.griddata(points, U.reshape((-1)), ([p[0]], [p[1]]), method='cubic')
        v_interp = interpolate.griddata(points, V.reshape((-1)), ([p[0]], [p[1]]), method='cubic')
        return np.array([u_interp[0], v_interp[0]])
    

    x_step = shelf_l + 1e-2
    y_step = shelf_w + passage_width 

    is_first_shelf = True
    while True:
        eigen_major = _get_major_eigen(cur_position)

        
        if is_horizontal(eigen_major, thresh):
            if np.random.rand() > skip_shelf_prob:
                shelf = Shelf(x=cur_position[0], y=cur_position[1], w=shelf_w, l=shelf_l, occupancy_width=occupancy_width)
                if shelf.is_valid(tf.N - 1, tf.M - 1) and not check_collisions(shelf, shelves):
                    shelves.append(shelf)
                    is_first_shelf = False
        
        x_step = 0.1 if is_first_shelf else shelf_l + 1e-2
        
        if cur_position[0] + x_step < tf.N:
            cur_position[0] += x_step
        else:
            if cur_position[1] + y_step < tf.M:
                cur_position[0] = start_point[0]
                cur_position[1] += y_step
            else:
                break

    cur_position = start_point.copy()
    x_step = shelf_w + passage_width 
    y_step = shelf_l + 1e-2 


    is_first_shelf = True
    while True:
        eigen_major = _get_major_eigen(cur_position)
        
        if is_vertical(eigen_major, thresh):
            if np.random.rand() > skip_shelf_prob:
                shelf = Shelf(x=cur_position[0], y=cur_position[1], w=shelf_w, l=shelf_l, orientation = 'vertical', occupancy_width=occupancy_width)
                if shelf.is_valid(tf.N - 1, tf.M - 1) and not check_collisions(shelf, shelves):
                    shelves.append(shelf)
                    is_first_shelf = False
                    
        y_step = 0.1 if is_first_shelf else shelf_l + 1e-2 
        
        if cur_position[1] + y_step < tf.M:
            cur_position[1] += y_step
        else:
            if cur_position[0] + x_step < tf.N:
                cur_position[1] = start_point[1]
                cur_position[0] += x_step
            else:
                break
    
    return shelves
    

In [ ]:
tf = TensorField(12, 8)
boundary = np.array([[0, 0], [0, tf.M], [tf.N, tf.M], [tf.N, 0],])
tf.add_line(boundary, closed=True, sample_step=1.)
tf.calculate_field(decay=12.)
tf.vis_field()
plt.show()

In [ ]:
shelves = place_shelves(tf, skip_shelf_prob=0.0, start_point=np.array([1.2, 1.2]), passage_width=0.7, occupancy_width=0.5)
fig, ax = tf.vis_field()
for shelf in shelves:
    shelf.draw(ax[1], show_occupancy=True)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_xlim(0, tf.N-1)
ax.set_ylim(0, tf.M-1)
# ax.axis('off')
ax.set_aspect('equal', adjustable='box')
for shelf in shelves:
    shelf.draw(ax, show_occupancy=False)